In [1]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Programa 2 para pegar movimentações de processos na busca do 1º Grau 
# Com o arquivo gerado no programa 1 entra em cada página de processo
# e capta informações iniciais do processo
#

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import urllib3; urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import time
import pandas as pd

In [2]:
df_processos = pd.read_excel('resultados/processos_teste_tjsp_22_abril.xlsx', Sheet="Sheet1")

In [3]:
df_processos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097 entries, 0 to 1096
Data columns (total 8 columns):
Unnamed: 0    1097 non-null int64
process       1097 non-null object
link          1097 non-null object
type          1097 non-null object
label         1097 non-null object
name          1097 non-null object
date          1097 non-null datetime64[ns]
location      1097 non-null object
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 68.6+ KB


In [11]:
processos = []

In [12]:
for num, row in df_processos.iterrows():
    link = row['link']
    num_processo = row['process']
    tipo = row['type']
    parte_especie = row['label']
    nome_parte = row['name']
    data = row['date']
    foro = row['location']
    
    time.sleep(1)
    
    r = requests.get(link, verify=False)
    
    soup = bs(r.content, 'lxml')
    
    assunto = soup.select_one('td:has(>.labelClass:contains("Assunto:")) + td').text.strip() if soup.select_one('td:has(>.labelClass:contains("Assunto:")) + td') is not None else 'N/A'
    
    mais_detalhes = soup.select_one('td:has(>.labelClass:contains("Outros assuntos:")) + td').text.strip() if soup.select_one('td:has(>.labelClass:contains("Outros assuntos:")) + td') is not None else 'N/A'
    
    data_distribuicao = soup.select_one('td:has(>.labelClass:contains("Distribuição:")) + td').text.strip() if soup.select_one('td:has(>.labelClass:contains("Distribuição:")) + td') is not None else 'N/A'
    
    magistrado = soup.select_one('td:has(>.labelClass:contains("Juiz:")) + td').text.strip() if soup.select_one('td:has(>.labelClass:contains("Juiz:")) + td') is not None else 'N/A'
    
    valor_acao = soup.select_one('td:has(>.labelClass:contains("Valor da ação:")) + td').text.strip() if soup.select_one('td:has(>.labelClass:contains("Valor da ação:")) + td') is not None else 'N/A'
    
    
    processos.append({'num_processo': num_processo,
                      'link': link,
                      'tipo': tipo,
                      'parte_especie': parte_especie,
                      'nome_parte': nome_parte,
                      'data': data,
                      'foro': foro,
                      'assunto': assunto,
                      'mais_detalhes': mais_detalhes,
                      'data_distribuicao': data_distribuicao,
                      'magistrado': magistrado,
                      'valor_acao': valor_acao
                     }
                      )    
    

In [13]:
df_processos_mais = pd.DataFrame(processos, columns = ['num_processo', 
                                                       'link', 
                                                       'tipo',
                                                       'parte_especie',
                                                       'nome_parte',
                                                       'data',
                                                       'foro', 
                                                       'assunto', 
                                                       'mais_detalhes', 
                                                       'data_distribuicao',
                                                       'magistrado', 
                                                       'valor_acao'])

In [14]:
df_processos_mais.reset_index().head(10)

,index,num_processo,link,tipo,parte_especie,nome_parte,data,foro,assunto,mais_detalhes,data_distribuicao,magistrado,valor_acao
0,0,1005696-11.2018.8.26.0053,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Anulação de Débito Fiscal,Reqte,Empresa Folha da Manhã S.A.,2018-06-02,Vara de Fazenda Pública,Anulação de Débito Fiscal,N/A,07/02/2018 às 11:18 - Livre,SIMONE GOMES RODRIGUES CASORETTI,"R$ 1.350.840,38"
1,1,1029643-16.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Intimação / Notificação,Reqda,Empresa Folha da Manhã S.A.,2019-03-04,Vara Cível,Intimação / Notificação,N/A,03/04/2019 às 13:03 - Livre,LUIZ ANTONIO CARRER,"R$ 1.000,00"
2,2,1028972-90.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Indenização por Dano Moral,Reqte,Empresa Folha da Manhã S.A.,2019-01-04,Vara Cível,Indenização por Dano Moral,"Perdas e Danos,Rescisão / Resolução",01/04/2019 às 19:13 - Livre,Lúcia Caninéo Campanhã,"R$ 214.136,38"
3,3,1020778-04.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Classificação de créditos,Reqte,Empresa Folha da Manhã S.A.,2019-11-03,Vara de Falências e Recuperações Judiciais,Classificação de créditos,N/A,11/03/2019 às 21:17 - Dependência (1110406-38....,MARCELO BARBOSA SACRAMONE,"R$ 18.114,03"
4,4,1004643-14.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Indenização por Dano Moral,Reqda,Empresa Folha da Manhã S.A.,2019-01-23,Vara Cível,Indenização por Dano Moral,N/A,23/01/2019 às 16:24 - Livre,Valéria Longobardi,"R$ 30.000,00"
5,5,1110406-38.2018.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Concurso de Credores,Credor,Empresa Folha da Manhã S.A.,2018-10-25,Vara de Falências e Recuperações Judiciais,Concurso de Credores,N/A,25/10/2018 às 12:11 - Livre,MARCELO BARBOSA SACRAMONE,"R$ 285.391.594,66"
6,6,1103409-39.2018.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Indenização por Dano Moral,Reqda,Empresa Folha da Manhã S.A.,2018-04-10,Vara Cível,Indenização por Dano Moral,Direito de Imagem,04/10/2018 às 22:07 - Livre,Guilherme Madeira Dezem,"R$ 60.000,00"
7,7,1099576-13.2018.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Dever de Informação,Reqda,Empresa Folha da Manhã S.A.,2018-09-25,Vara Cível,Dever de Informação,N/A,25/09/2018 às 14:16 - Livre,FABIO DE SOUZA PIMENTA,"R$ 50.000,00"
8,8,1084733-43.2018.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Concurso de Credores,Credor,Empresa Folha da Manhã S.A.,2018-08-15,Vara de Falências e Recuperações Judiciais,Concurso de Credores,N/A,16/08/2018 às 00:07 - Livre,PAULO FURTADO DE OLIVEIRA FILHO,"R$ 1.636.979.952,73"
9,9,1078123-59.2018.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Indenização por Dano Moral,Reqda,Empresa Folha da Manhã S.A.,2018-07-30,Vara Cível,Indenização por Dano Moral,"Direito de Imagem,Lei de Imprensa",30/07/2018 às 10:02 - Livre,ANDRE LUIZ DA SILVA DA CUNHA,"R$ 50.000,00"


In [15]:
df_processos_mais.to_excel('resultados/processos_teste_tjsp_23_abril_segunda_fase.xlsx',sheet_name='Sheet1')

In [16]:
df_processos_mais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097 entries, 0 to 1096
Data columns (total 12 columns):
num_processo         1097 non-null object
link                 1097 non-null object
tipo                 1097 non-null object
parte_especie        1097 non-null object
nome_parte           1097 non-null object
data                 1097 non-null datetime64[ns]
foro                 1097 non-null object
assunto              1097 non-null object
mais_detalhes        1097 non-null object
data_distribuicao    1097 non-null object
magistrado           1097 non-null object
valor_acao           1097 non-null object
dtypes: datetime64[ns](1), object(11)
memory usage: 102.9+ KB
